## TO DO:

1. Modify the parser to pull the entire story text instead of just the parts tagged as `<p>`, so it'll pull Miranda's stuff as well.
2. Run scraper for first 8,000 URLs, since it keeps getting hung up for whatever reason.
3. Put the 8,000 results into a DF and then use Python magic to figure out which URLs need scraped, then get the rest of them.
4. Pester A3 for all of 2020's URLs.

## Imports

In [6]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd
from IPython.display import clear_output

## Set Options

In [137]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 1000) ### Default
pd.set_option('display.max_colwidth', 100) ### Default 50

## Data Read-in

<em>All</em> Chicago URLs

In [3]:
df = pd.read_csv('Chicago URLS by year - 2017.csv',header=None)
df2 = pd.read_csv('Chicago URLS by year - 2018.csv',header=None)
df3 = pd.read_csv('Chicago URLS by year - 2019.csv',header=None)
df4 = pd.read_csv('Chicago URLS by year - 2020.csv',header=None)
df5 = pd.read_csv('Chicago URLS by year - 2021.csv',header=None)
df6 = pd.read_csv('Chicago URLS by year - 2022.csv',header=None)

In [4]:
frames = [df,df2,df3,df4,df5,df6]
df = pd.concat(frames)
df = df.reset_index(drop=True)
df = df.rename(columns={0:'URL'})

In [8]:
len(df)

12222

In [9]:
# df1 = df.iloc[:7000,:]
# df2 = df.iloc[7000:,:]

In [36]:
# print(len(df1))
# print('------')
# print(len(df2))

In [11]:
# ### Original Data set - kept for backup
# df = pd.read_csv('TRD_CHI-Coverage-Valid-2022-12-01 - Table.csv')
# df = df.drop(columns='Last crawled')

## Test Ground

In [12]:
# page = requests.get('https://therealdeal.com/2022/12/08/adams-announces-moonshot-goal-500k-new-homes/?utm_source=internal&utm_medium=article_aside&utm_campaign=trd_list_posts&utm_content=trending_now')
# soup = BeautifulSoup(page.content, 'html.parser')

In [13]:
# soup.find('h1').text.strip() ### Title
# soup.find('h2').text.strip() ### Subhead
# soup.find('div', class_='col-7 col-md-3 order-md-3 text-right').text.strip() ### Pub_Date
# soup.find_all('p')[2].text.strip() ### Lede

In [14]:
# story_text = soup
# story_text_clean = []
# for p in story_text:
#     p = p.text.strip()
#     story_text_clean.append(p)
# story_text_clean = [''.join(story_text_clean)]
# story_text_clean

In [15]:
# story_headlines = soup.find_all('h3')
# story_headlines_clean = []
# for h3 in story_headlines:
#     h3 = h3.text.strip()
#     story_headlines_clean.append(h3)
# story_headlines_clean = [''.join(story_headlines_clean)]

In [16]:
# story_text = soup.find_all('p')[2:]
# story_text_clean = []
# for p in story_text:
#     p = p.text.strip()
#     story_text_clean.append(p)
# story_text_clean = [''.join(story_text_clean)]

TO DO: Fix image scraper to get PNGs...or succesfull append something

In [17]:
# images = soup.find_all('img')
# for image in images:
#     if 'jpg' in image['src']:
#         print(image['src'])

In [18]:
# images = soup.find_all('img')
# png_list = []
# for image in images:
#     if 'png' in image['src']:
#         png_list.append(image['src'])
# print(png_list[-1])

## Scraper

In [19]:
### Create Lists

url_list = []
hed_list = []
dek_list = []
pub_date_list = []
lede_list = []
story_text_list = []
story_headline_list = []

In [20]:
counter = 1
for url in df['URL']:
    if counter > 13000:
        break
    else:
        try:
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            counter = counter + 1
            clear_output(wait=True)
            url_list.append(url)
            print(counter)
            print(url)
            
            ####################
            # HED #
            
            try:
                hed_1 = soup.find('h1').text.strip()
                hed_list.append(hed_1)
            except Exception as e:
                print(f'HED fail {url} ERROR: {e}')
                hed_list.append(f'HED FAIL {url} ERROR: {e}')
            
            ####################
            # DEK #
            
            try:
                dek_1 = soup.find('h2').text.strip()
                dek_list.append(dek_1)
            except Exception as e:
                print(f'DEK fail {url} ERROR: {e}')
                dek_list.append(f'DEK FAIL {url} ERROR: {e}')
                
            ####################  
            # PUB_DATE #
            
            try:
                pub_date_1 = soup.find('div', class_='col-7 col-md-3 order-md-3 text-right').text.strip()
                pub_date_list.append(pub_date_1)
            except Exception as e:
                print(f'PUB_DATE fail {url} ERROR: {e}')
                pub_date_list.append(f'PUB_DATE FAIL {url} ERROR: {e}')
            
            ####################
            # LEDE #
            
            try:  
                lede_1 = soup.find_all('p')[2].text.strip()
                lede_list.append(lede_1)
            except Exception as e:
                print(f'LEDE fail {url} ERROR: {e}')
                lede_list.append(f'LEDE FAIL {url} ERROR: {e}')
                
            ####################
            # STORY TEXT & H3 headlines #
                
            try:
                story_text = soup.find_all('p')[2:]
                story_text_clean = []
                for p in story_text:
                    p = p.text.strip()
                    story_text_clean.append(p)
                story_text_clean = [''.join(story_text_clean)]
                story_text_list.append(story_text_clean)
                
            except Exception as e:
                print(f'STORY TEXT fail {url} ERROR: {e}')
                story_text_list.append(f'STORY TEXT FAIL {url} ERROR: {e}')
                
            try:
                story_headlines = soup.find_all('h3')
                story_headlines_clean = []
                for h3 in story_headlines:
                    h3 = h3.text.strip()
                    story_headlines_clean.append(h3)
                story_headlines_clean = [''.join(story_headlines_clean)]
                story_headline_list.append(story_headlines_clean)
                
            except Exception as e:
                print(f'STORY H3 fail {url} ERROR: {e}')
                story_headline_list.append(f'STORY H3 FAIL {url} ERROR: {e}')

        except Exception as e:
            print(f'{url} - FAIL: {e}')
            url_list.append(f'{url} - FAIL: {e}')
            hed_list.append(f'{url} - FAIL: {e}')
            dek_list.append(f'{url} - FAIL: {e}')
            pub_date_list.append(f'{url} - FAIL: {e}')
            lede_list.append(f'{url} - FAIL: {e}')
            story_text_list.append(f'{url} - FAIL: {e}')
            story_headline_list.append(f'{url} - FAIL: {e}')

12221
https://therealdeal.com/chicago/?p=381293
DEK fail https://therealdeal.com/chicago/?p=381293 ERROR: 'NoneType' object has no attribute 'text'
PUB_DATE fail https://therealdeal.com/chicago/?p=381293 ERROR: 'NoneType' object has no attribute 'text'


In [21]:
print(len(url_list))
print('---------')
print(len(hed_list))
print('---------')
print(len(dek_list))
print('---------')
print(len(pub_date_list))
print('---------')
print(len(lede_list))
print('---------')
print(len(story_text_list))
print('---------')
print(len(story_headline_list))
print('---------')

12222
---------
12222
---------
12222
---------
12222
---------
12222
---------
12222
---------
12222
---------


In [22]:
df['URL_2'] = url_list
df['HED'] = hed_list
df['DEK'] = dek_list
df['PUB_DATE'] = pub_date_list
df['LEDE'] = lede_list
df['STORY_TEXT'] = story_text_list
df['STORY_H3_HEADLINES'] = story_headline_list

## Parser Playground

In [126]:
df['STORY_TEXT'] = df['STORY_TEXT'].astype(str)

In [29]:
# df['URL'].iloc[20]

In [158]:
df['ADDRESS'] = df['STORY_TEXT'].str.extract(r'(?P<ADDRESS>[1-9][0-9]* (?:\w+\W+){1,6}(?:Avenue|Ave|Boulevard|Blvd|Broadway|Court|Drive|Dr|Huron|Place|Road|Rd|Street|St))')
df['ADDRESS'].isna().value_counts()

True     7708
False    4514
Name: ADDRESS, dtype: int64

In [162]:
# df['STORY_TEXT'].iloc[236]

In [165]:
# x = 11000
# y = x + 1000
# df[['URL','ADDRESS','HED']].iloc[x:y]

In [33]:
# df = df.drop(columns='SOUP')

In [34]:
# df.to_csv('Chicago_sample.csv')

## Link Formatter

In [166]:
df["short_description"] = df["LEDE"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['LEDE'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["URL"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
# df['description_link']

In [167]:
df['PUB_YEAR'] = df['PUB_DATE'].str.extract(r'(\d{4})')

## Geocoder Setup

In [168]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [169]:
df['geo_address'] = df['ADDRESS'] + ' Chicago, IL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [174]:
df.to_json('Chicago_story_map_data.json')

In [175]:
df.columns

Index(['URL', 'URL_2', 'HED', 'DEK', 'PUB_DATE', 'LEDE', 'STORY_TEXT',
       'STORY_H3_HEADLINES', 'ADDRESS', 'short_description', 'remaining_desc',
       'description_link', 'PUB_YEAR', 'geo_address', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [240]:
df2 = df.drop(columns=['STORY_H3_HEADLINES','PUB_DATE','STORY_TEXT','short_description','LEDE','remaining_desc'])
df2 = df2.dropna(subset='loc')
df2.drop(df2.loc[df2['DEK'].str.contains('fail')].index, inplace=True)
df2.drop(df2.loc[df2['HED'].str.contains('fail')].index, inplace=True)
df2.reset_index(inplace=True)
df2 = df2.drop(columns='index')

In [251]:
# from pandas_geojson import to_geojson

In [221]:
df2.columns

Index(['URL', 'URL_2', 'HED', 'DEK', 'ADDRESS', 'description_link', 'PUB_YEAR',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [218]:
# df2['URL_2'].str.contains('FAIL').value_counts()

False    4336
Name: URL_2, dtype: int64

In [250]:
df2.to_csv('Chicago_story_map_data.csv', sep='|')
df2.to_json('Chicago_story_map_data.json')

In [252]:
df2

,URL,URL_2,HED,DEK,ADDRESS,...,loc,point,lat,lon,altitude
0,https://therealdeal.com/chicago/?p=317106,https://therealdeal.com/chicago/?p=317106,Real estate’s diversity problem,An entrenched culture marked by privately held family firms has largely blocked out minorities a...,108 Leonard Street,...,"(N Leonard Ave, Chicago, IL, USA, (41.9872007, -87.7682537))","(41.9872007, -87.7682537, 0.0)",41.987201,-87.768254,0.0
1,https://therealdeal.com/chicago/?p=350083,https://therealdeal.com/chicago/?p=350083,An insider’s guide to real estate syndication,"How the popular investment model works, and who you need to know to get it done",432 Park Avenue,...,"(N North Park Avenue, Chicago, IL, USA, (41.9106866, -87.636645))","(41.9106866, -87.636645, 0.0)",41.910687,-87.636645,0.0
2,https://therealdeal.com/chicago/?p=302451,https://therealdeal.com/chicago/?p=302451,Highgate: The quiet hotel powerhouse,"As investors in the sector retreat, Highgate has doubled down, becoming Manhattan’s most active ...",650 Madison Avenue,...,"(650 W Madison St, Chicago, IL 60661, USA, (41.8821492, -87.64469369999999))","(41.8821492, -87.64469369999999, 0.0)",41.882149,-87.644694,0.0
3,https://therealdeal.com/chicago/?p=122962,https://therealdeal.com/chicago/?p=122962,Flagship Apple store on North Michigan could fetch up to $175M: report,Walton Street Capital wants to unload the retail portion of its office tower,401 North Michigan Avenue,...,"(401 N Michigan Ave, Chicago, IL 60606, USA, (41.88955600000001, -87.6230294))","(41.88955600000001, -87.6230294, 0.0)",41.889556,-87.623029,0.0
4,https://therealdeal.com/chicago/?p=123010,https://therealdeal.com/chicago/?p=123010,Drugmaker Depomed shifts headquarters to Lake Forest,The new space is in an office park alongside other pharmaceutical firms,100 South Saunders Road,...,"(100 Saunders Rd, Lake Forest, IL 60045, USA, (42.238585, -87.8989811))","(42.238585, -87.8989811, 0.0)",42.238585,-87.898981,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4309,https://therealdeal.com/chicago/?p=382415,https://therealdeal.com/chicago/?p=382415,Glencoe’s priciest home this year could be demolished for new digs,Mansion sold for full ask in less than a week on the market,333 Surfside Place,...,"(Chicago, IL, USA, (41.8781136, -87.6297982))","(41.8781136, -87.6297982, 0.0)",41.878114,-87.629798,0.0
4310,https://therealdeal.com/chicago/?p=382448,https://therealdeal.com/chicago/?p=382448,Joint venture on South Loop mixed-use tower gets OK’d,Tower will include 164 resi units,626 South Wabash Avenue,...,"(626 S Wabash Ave, Chicago, IL 60605, USA, (41.8737364, -87.62597570000001))","(41.8737364, -87.62597570000001, 0.0)",41.873736,-87.625976,0.0
4311,https://therealdeal.com/chicago/?p=382463,https://therealdeal.com/chicago/?p=382463,Vista Property plans another West Loop office project,New York developer has multiple projects ongoing across downtown,640 West Randolph Street,...,"(640 W Randolph St, Chicago, IL 60661, USA, (41.8848249, -87.6446948))","(41.8848249, -87.6446948, 0.0)",41.884825,-87.644695,0.0
4312,https://therealdeal.com/chicago/?p=382514,https://therealdeal.com/chicago/?p=382514,Will new Board of Review commissioners quell feud with Kaegi?,"Samantha Steele, George Cardenas join Larry Rogers Jr.",20 years while St,...,"(Chicago, IL, USA, (41.8781136, -87.6297982))","(41.8781136, -87.6297982, 0.0)",41.878114,-87.629798,0.0


In [171]:
df[['geo_address','lat','lon','altitude']]

,geo_address,lat,lon,altitude
0,"108 Leonard Street Chicago, IL",41.987201,-87.768254,0.0
1,NaN,NaN,NaN,NaN
2,"300 South Grand Avenue Chicago, IL",41.931535,-87.929152,0.0
3,"432 Park Avenue Chicago, IL",41.910687,-87.636645,0.0
4,NaN,NaN,NaN,NaN
...,...,...,...,...
12217,"20 years while St Chicago, IL",41.878114,-87.629798,0.0
12218,NaN,NaN,NaN,NaN
12219,NaN,NaN,NaN,NaN
12220,"1909 North Orchard Street Chicago, IL",41.916400,-87.645672,0.0


## Code graveyard

In [35]:
            ####################
            # IMAGES #
            
#             try:            
#                 images = soup.find_all('img')
#                 image_found = False
#                 try:
#                     while image_found == False:
#                         png_list = []
#                         for image in images:
#                             if 'jpg' in image['src']:
#                                 image_list.append(image['src'])
#                                 print(image['src'])
#                                 image_found = True
#                             break
#                         if image_found == False:
#                             for image in images:
#                                 if 'png' in image['src']:
#                                     png_list.append(image['src'])
#                                     image_found = True
#                             image_list.append(png_list[-1])
#                             print(png_list[-1])
#                 except Exception as e:
#                     print(f'No dice: {e}')
#                 print('------')
#             except Exception as e:
#                 print('find_image_fail')
            
            ####################

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [1]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [8]:
cwd = 'chicago_transaction_scraper_map'

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chicago_transaction_scraper_map
